#mongodb
1. 라이브러리 설치
2. 라이브러리 로드
3. CRUD

In [1]:
!pip install pymongo

     -------------------------------------- 382.2/382.2 kB 5.9 MB/s eta 0:00:00
     -------------------------------------- 283.7/283.7 kB 8.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient(
    host = 'localhost',
    port = 27017
)

In [4]:
# mongodb에 있는 데이터베이스의 목록 출력
client.list_database_names()

['admin', 'config', 'local']

In [6]:
# 데이터베이스를 선택
db = client['ubion'] #없지만 실행됨

In [7]:
client.list_database_names() #'ubion'이 생기긴 했지만 아무것도 들어있지 않아서 안 뜸

['admin', 'config', 'local']

In [8]:
# collection 접근(collection 은 sql에서 table 과 같은 의미)
collection = db['test'] #없지만 실행됨

In [9]:
# CRUD
# create -> insert()
# insert_one() /insert_many()
data = {
    'id':'test',
    'pass': '1234'
}

collection.insert_one(data)

In [10]:
collection.find_one()
# 결과가 '_id': ObjectId('643f8ae6020c96cffb922318') 이 뜻은 primary key

{'_id': ObjectId('643f8ae6020c96cffb922318'), 'id': 'test', 'pass': '1234'}

In [12]:
#mongodb의 장점. 컬럼이 다른데 생성가능
data = [
    {
        'name':'test1',
        'phone': '01011112222',
        'gender': 'male'
    },
    {
        'loc': 'seoul'
    }
]

collection.insert_many(data)

In [13]:
list(collection.find())

[{'_id': ObjectId('643f8ae6020c96cffb922318'), 'id': 'test', 'pass': '1234'},
 {'_id': ObjectId('643f8ccf020c96cffb922319'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8ccf020c96cffb92231a'), 'loc': 'seoul'}]

In [14]:
client.list_database_names()

['admin', 'config', 'local', 'ubion']

1. sql에서 test table에서 name이 test1인 경우의 데이터를 출력
    - select * from test where name = 'test1'

In [15]:
list(collection.find({'name':'test1'}))

[{'_id': ObjectId('643f8ccf020c96cffb922319'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'}]

1. 해당하는 컬럼의 데이터만 출력하려면
    - select name from testm

In [16]:
list(collection.find({}, {'name':1}))

[{'_id': ObjectId('643f8ae6020c96cffb922318')},
 {'_id': ObjectId('643f8ccf020c96cffb922319'), 'name': 'test1'},
 {'_id': ObjectId('643f8ccf020c96cffb92231a')}]

# update
1. 데이터의 수정
    - update test set pass = '0000' where id = 'test'
    - test 테이블에 id가 test인 데이터에서 pass의 값을 0000으로 수정하겠다.

In [17]:
collection.update_one(
    {
        'id': "test"
    },
    {
        "$set" : {
            'pass': '0000'
        }
    }
)

In [18]:
collection.find_one()

{'_id': ObjectId('643f8ae6020c96cffb922318'), 'id': 'test', 'pass': '0000'}

1. 데이터의 삭제
    - delete from test where id = 'test'
    - test 테이블에서 id가 'test'인 데이터를 삭제

In [19]:
collection.delete_one(
    {
        'id':'test'
    }
)

In [20]:
list(collection.find())

[{'_id': ObjectId('643f8ccf020c96cffb922319'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8ccf020c96cffb92231a'), 'loc': 'seoul'}]

In [21]:
list(client['ubion']['test'].find())

[{'_id': ObjectId('643f8ccf020c96cffb922319'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8ccf020c96cffb92231a'), 'loc': 'seoul'}]

### mongodb select 조건식

- find(옵션1, 옵션2, 옵션3, ..., 컬럼 선택)

    1. db.collection.find() -> select * from TABLE
    2. db.collection.find({}, {"id":1 , "pass":1}) -> select id, pass from TABLE
    3. db.collection.find({}, {"id":1, "pass":1, "_id": 0}) -> 2번 과정과 같은 sql문이지만 _id를 보여주지 않겠다.
    4. db.collection.find({"id":"test"}) -> select * from TABLE where id = "test"
    5. db.collection.find({"id":"test", "pass":"1234"}) -> select * from TABLE where id='test' and pass='1234'
    6. db.collection.find({"id":"test"}, {"pass":"1234"}) -> select * from TABLE where id='test' or pass='1234'
    7. db.collection.find({'age': {"$gt" : 25}}) -> select * from TABLE where age > 25
    8. db.collection.find({'age': {"$lt" : 25}}) -> select * from TABLE where age < 25
    9. db.collection.find({'id': {/bc/}}) ->  select * from TABLE where id like '%bc%'
    10. db.collection.find().sort({"id":1}) -> select * from TABLE order by id ASC
    11. db.collection.find().sort({"id":-1}) -> select * from TABLE order by id DESC
    12. db.collection.find().limit(10) -> select * from TABLE limit 10